<a href="https://colab.research.google.com/github/CPJKU/beat_this/blob/main/beat_this_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beat This! inference example

We first need to install and load the package.

In [ ]:
# install the beat_this package
!pip install https://github.com/CPJKU/beat_this/archive/main.zip

# load the Python class for beat tracking
from beat_this.inference import File2Beats
from beat_this.inference import File2File

## Run on demo file

Now that all the dependencies have been installed and imported, let's run our system.

In the next cell we:
- define the audio file we want to use as input. For now we use the example provided in the beat_this repo, but this can be changed (see instructions later);
- load the File2Beats class that produce a list of beats and downbeats given an audio file;
- apply the class to the audio file
- print the position in seconds of the first 20 beats and first 20 downbeats.


In [ ]:
!wget -c "https://github.com/CPJKU/beat_this/raw/main/tests/It%20Don't%20Mean%20A%20Thing%20-%20Kings%20of%20Swing.mp3"
audio_path = "/content/It Don't Mean A Thing - Kings of Swing.mp3"

file2beats = File2Beats(checkpoint_path="final0", dbn=False)
beats, downbeats = file2beats(audio_path)

print("First 20 beats", beats[:20])
print("First 20 downbeats", downbeats[:20])

We can sonify the beats and downbeats as click on top of the audio file.

In [ ]:
import IPython.display as ipd
import librosa
import numpy as np
import soundfile as sf

audio, sr = sf.read(audio_path)
# make it mono if stereo
if len(audio.shape) > 1:
  audio = np.mean(audio, axis=1)

# sonify the beats and downbeats
# remove the beats that are also downbeats for a nicer sonification
beats = [b for b in beats if b not in downbeats]
audio_beat = librosa.clicks(times = beats, sr=sr, click_freq=1000, length=len(audio))
audio_downbeat = librosa.clicks(times = downbeats, sr=sr, click_freq=1500, length=len(audio))

ipd.display(ipd.Audio(audio + audio_beat + audio_downbeat, rate=sr))

## Run on your own file

If you want to run on your own audio files follow the following instructions:
1. Click on the folder icon in the left vertical menu.
2. Click on the "Upload to session storage" icon with the upward pointing arrow.

    This will add an audio file to the current colab runtime (it could take some time, and you may need to refresh the file manager using the dedicated button to see the new file). You can copy the audio path by clicking on the three dots next to the file, then "copy path".

    For example, if you upload a file called `my_song.mp3`, the path will be `/content/my_song.mp3`.

3. change the `audio_path` in the cell above with the path of your uploaded audio

You can also produce a list of beat and downbeat as tsv file, that you can download and import in Sonic Visualizer.

To do this this, use the File2File function as below:

In [ ]:
file2file = File2File(checkpoint_path="final0", dbn=False)
file2file(audio_path,output_path="output.beats")

As you can see, the system is fast enough to work in a reasonable time even on CPU.

For even faster inference, you can start a GPU session in Colab!

## Batch processing multiple files

To process multiple of your own audio files, upload them as described above, then run the `beat_this` command line tool:

In [ ]:
!beat_this --model final0 /content/

It will produce a `.beats` file for every audio file that you can download again.